<a href="https://colab.research.google.com/github/SalarShafiee/Image-classification/blob/main/Tomato-Image%20data%20generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"salarshafiee71","key":"702b6e7e2080f71cd323658809f86ee7"}'}

In [4]:
! mkdir ~/.kaggle

In [5]:
!kaggle datasets download -d noulam/tomato -p /content

Dataset URL: https://www.kaggle.com/datasets/noulam/tomato
License(s): copyright-authors
100% 741M/743M [00:45<00:00, 16.8MB/s]
100% 743M/743M [00:45<00:00, 17.2MB/s]


In [6]:
!unzip -q /content/tomato.zip -d /content/tomato

In [7]:
!nvidia-smi

Sun Jul 14 16:24:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model  # Corrected 'model' to 'Model'
# from tensorflow.keras.applications.inception_v3 import InceptionV3  # Example commented out
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob  # Corrected a typo: 'glob.' to 'glob'

In [10]:
image_size=[224,224]
train_path='/content/tomato/New Plant Diseases Dataset(Augmented)/train'
valid_path='/content/tomato/New Plant Diseases Dataset(Augmented)/valid'

In [12]:
VGG16=VGG16(input_shape=image_size+[3],weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 4s 0us/step


In [13]:
for layers in VGG16.layers:
  layers.trainable=False

In [17]:
folders=glob('/content/tomato/New Plant Diseases Dataset(Augmented)/train/*')
folders

['/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Target_Spot',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Leaf_Mold',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_mosaic_virus',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Bacterial_spot',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Spider_mites Two-spotted_spider_mite',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Early_blight',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Late_blight',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___healthy',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Septoria_leaf_spot']

In [20]:
x=Flatten()(VGG16.output)
prediction=Dense(len(folders),activation='softmax')(x)

In [21]:
model=Model(inputs=VGG16.input, outputs=prediction)

In [22]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [24]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='categorical_crossentropy')

In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [27]:
training_set=train_datagen.flow_from_directory(train_path,target_size=(224,224),batch_size=16,class_mode='categorical')

Found 18345 images belonging to 10 classes.


In [28]:
test_set=train_datagen.flow_from_directory(valid_path,target_size=(224,224),batch_size=16,class_mode='categorical')

Found 4585 images belonging to 10 classes.


In [29]:
history=model.fit_generator(training_set,validation_data=test_set,epochs=10)

<ipython-input-29-2f334b29a5f8>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(training_set,validation_data=test_set,epochs=10)


Epoch 1/10
1147/1147 [==============================] - 343s 293ms/step - loss: 0.8094 - accuracy: 0.7469 - val_loss: 0.6599 - val_accuracy: 0.7952
Epoch 2/10
1147/1147 [==============================] - 335s 292ms/step - loss: 0.5130 - accuracy: 0.8438 - val_loss: 0.4506 - val_accuracy: 0.8595
Epoch 3/10
1147/1147 [==============================] - 317s 277ms/step - loss: 0.4802 - accuracy: 0.8597 - val_loss: 0.5597 - val_accuracy: 0.8449
Epoch 4/10
1147/1147 [==============================] - 312s 272ms/step - loss: 0.3880 - accuracy: 0.8878 - val_loss: 0.4344 - val_accuracy: 0.8820
Epoch 5/10
1147/1147 [==============================] - 313s 273ms/step - loss: 0.3730 - accuracy: 0.8945 - val_loss: 0.6064 - val_accuracy: 0.8526
Epoch 6/10
1147/1147 [==============================] - 311s 271ms/step - loss: 0.3387 - accuracy: 0.9078 - val_loss: 0.5092 - val_accuracy: 0.8739
Epoch 7/10
1147/1147 [==============================] - 331s 288ms/step - loss: 0.3394 - accuracy: 0.9079 - val_